# Laboratorio 7

1. Utilice los juicios de relevancia de la colección CACM almacenados en el archivo qrels.txt
1. Utilice la salida de la práctica 6 para calcular Precisión, Recuerdo, Media F, Precisión R a Z documentos para cada una de las consultas. Z será un valor leído de pantalla donde se establecerá a que número de documentos deberán calcularse la métricas solicitadas.
1. Guarde sus resultados en un archivo con Z = 50 en el formato siguiente:

|  Consulta |Precisión|Recuerdo| Medida F  | Precisión R  |
|:---:|:---:|:---:|:---:|:---:|
|  1    | 0.45  | 0.35  | 0.39  | 0.43 |
| 2     | 0.5   | 0.4   | 0.2   | 0.45 | 
    

---



Primero, importaremos el archivo qrels.text como un dataframe con pandas


In [1]:
import pandas as pd
rel_df = pd.read_csv("assets/qrels.text", sep=" ", header=None, usecols=[0,1], names=['Query_num','Doc_num'])
rel_df['Relevant'] = 1
print(rel_df)

     Query_num  Doc_num  Relevant
0            1     1410         1
1            1     1572         1
2            1     1605         1
3            1     2020         1
4            1     2358         1
..         ...      ...       ...
791         63     2714         1
792         63     2973         1
793         63     3075         1
794         63     3156         1
795         64     2651         1

[796 rows x 3 columns]


Ahora, importamos los juicios de coincidencia creados en la práctica 6

In [13]:

with open('assets\\CACM_TFIDF_rels.txt', 'r') as file:
    sim_df = pd.read_json(file)
# change names of sim_df
sim_df = sim_df.set_axis(['Query_num','Doc_num','Sim'], axis=1)
#sim_df['Relevant'] = 0
print(sim_df)


       Query_num  Doc_num       Sim
0              1     1938  0.412627
1              1     1071  0.392878
2              1     2371  0.385110
3              1     2319  0.311126
4              1     1572  0.298465
...          ...      ...       ...
66838         64     1194  0.019560
66839         64     1641  0.019515
66840         64     1470  0.019013
66841         64     2359  0.018085
66842         64     3032  0.017947

[66843 rows x 3 columns]


In [14]:
display(sim_df.loc[(sim_df.Query_num == 1) & (sim_df.Doc_num == 1410)].iloc[:50])
display(sim_df.loc[(sim_df.Query_num == 1) & (sim_df.Doc_num == 1572)].iloc[:50])
display(sim_df.loc[(sim_df.Query_num == 1) & (sim_df.Doc_num == 1605)].iloc[:50])
display(sim_df.loc[(sim_df.Query_num == 1) & (sim_df.Doc_num == 2020)].iloc[:50])
display(sim_df.loc[(sim_df.Query_num == 1) & (sim_df.Doc_num == 2358)].iloc[:50])

,Query_num,Doc_num,Sim
71,1,1410,0.163379


,Query_num,Doc_num,Sim
4,1,1572,0.298465


,Query_num,Doc_num,Sim
16,1,1605,0.215273


,Query_num,Doc_num,Sim
498,1,2020,0.066417


,Query_num,Doc_num,Sim
23,1,2358,0.205964


In [15]:
# contamos en df relevantes
cont_rel_df = pd.DataFrame( )
for i in range(1,64+1):
    cont_rel_df[i] = rel_df[rel_df.Query_num == i].count()
    #print(sim_df[sim_df.Query_num == i].count())
    
cont_rel_df.drop(['Doc_num','Relevant'], inplace=True)
cont_rel_df = cont_rel_df.T
cont_rel_df

,Query_num
1,5
2,3
3,6
4,12
5,8
...,...
60,27
61,31
62,8
63,12


In [16]:
# df relevantes recuperados
relRec_df_all = pd.merge(sim_df, rel_df, how='left', on=['Query_num','Doc_num'])
relRec_df = relRec_df_all.loc[relRec_df_all.Relevant == 1]
#relRec_df.drop(['Doc_num','Sim','Relevant'],inplace=True)
relRec_df

,Query_num,Doc_num,Sim,Relevant
4,1,1572,0.298465,1.0
16,1,1605,0.215273,1.0
23,1,2358,0.205964,1.0
71,1,1410,0.163379,1.0
498,1,2020,0.066417,1.0
...,...,...,...,...
65026,63,3075,0.134567,1.0
65030,63,1811,0.132342,1.0
65036,63,2266,0.124351,1.0
65040,63,2973,0.118186,1.0


In [17]:
# contamos df relevantes recuperados
# contamos en df relevantes
cont_relRec_df = pd.DataFrame( )
for i in range(1,64+1):
    cont_relRec_df[i] = relRec_df[relRec_df.Query_num == i].count()
    #print(sim_df[sim_df.Query_num == i].count())
    
cont_relRec_df.drop(['Doc_num','Sim','Relevant'], inplace=True)
cont_relRec_df = cont_relRec_df.T
cont_relRec_df

,Query_num
1,5
2,0
3,5
4,11
5,8
...,...
60,21
61,28
62,7
63,11


Ahora, con los conteos hechos, calculamos las estadísticas sobre cada query.

Todo se guardará sobre un nuevo dataframe

In [18]:
# antes, agrupamos para contar los primeros Z=50 records de cada recuperación
Z = 50
pres_R_df = pd.DataFrame()
relRec_df_all['Relevant'].fillna(0, inplace=True)
for i in range(1,64+1):
    temp = relRec_df_all[relRec_df_all['Query_num']==i].iloc[:Z]
    pres_R_df[i] = temp[temp['Relevant'] == 1].count()


In [19]:
#twenty = relRec_df_all[relRec_df_all['Query_num']==1].iloc[:20]
#
# twenty
pres_R_df.drop(['Doc_num','Sim','Relevant'], inplace=True)
pres_R_df = pres_R_df.T
pres_R_df


,Query_num
1,3
2,0
3,1
4,4
5,4
...,...
60,4
61,13
62,2
63,8


In [20]:
# precision:            #docs_rel_rec/#docs_recuperados
# recall (recuerdo):    #docs_rel_rec/#docs_rel
# F-score:              2PR/(P+R)
# Presicion R a Z:                  Precisión hasta el Z-ésimo documento recuperado (Z=20)

stats_df = pd.DataFrame({'Precision': pd.Series(dtype='float'), 
                         'Recall': pd.Series(dtype='float'),
                         'F-score': pd.Series(dtype='float'),
                         'Precision-R': pd.Series(dtype='float')})
for i in range(1,64+1):
    pres = (pres_R_df.at[i,'Query_num'])/ Z
    recall = (pres_R_df.at[i,'Query_num'])/ (cont_rel_df.at[i, 'Query_num'])
    if (pres + recall == 0):
        F_score = 0
    else:
        F_score = (2*pres*recall)/(pres+recall)
    
    R = (pres_R_df.at[i,'Query_num'])
    temp = relRec_df_all[relRec_df_all['Query_num']==i].iloc[:R]
    rel_rec_R = temp[temp['Relevant'] == 1].count().values[0]
    pres_R =  rel_rec_R / (pres_R_df.at[i,'Query_num'])
    stats_df.loc[len(stats_df.index)+1] = [pres, recall, F_score, pres_R]
stats_df.fillna(0, inplace=True)


C:\Users\poncho\AppData\Local\Temp\ipykernel_1652\385762732.py:21: RuntimeWarning: invalid value encountered in longlong_scalars
  pres_R =  rel_rec_R / (pres_R_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_1652\385762732.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (pres_R_df.at[i,'Query_num'])/ (cont_rel_df.at[i, 'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_1652\385762732.py:21: RuntimeWarning: invalid value encountered in longlong_scalars
  pres_R =  rel_rec_R / (pres_R_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_1652\385762732.py:12: RuntimeWarning: invalid value encountered in longlong_scalars
  recall = (pres_R_df.at[i,'Query_num'])/ (cont_rel_df.at[i, 'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykernel_1652\385762732.py:21: RuntimeWarning: invalid value encountered in longlong_scalars
  pres_R =  rel_rec_R / (pres_R_df.at[i,'Query_num'])
C:\Users\poncho\AppData\Local\Temp\ipykerne

In [21]:
# verificar datos
display(stats_df.iloc[10])
display(cont_rel_df.iloc[10])
display(pres_R_df.iloc[10])
display(cont_relRec_df.iloc[10])
display(pres_R_df.iloc[10])

Precision      0.220000
Recall         0.578947
F-score        0.318841
Precision-R    0.363636
Name: 11, dtype: float64

Query_num    19
Name: 11, dtype: int64

Query_num    11
Name: 11, dtype: int64

Query_num    16
Name: 11, dtype: int64

Query_num    11
Name: 11, dtype: int64

Por último, lo guardamos a un archivo

In [22]:
from pathlib import Path  
filepath = Path('results/stats.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
stats_df.to_csv(filepath) 
filepath = Path('results/stats.json')  
stats_df.to_json(filepath) 